#  EXTRACTING INFORMATION FOR OUR DATA SET USING SPOTIPY

In [1]:
pip install spotipy

In [2]:
import spotipy
import spotipy.util as util
from spotipy import oauth2
        
import pandas as pd

# Reading our credentials to access the API

In [3]:
#SPOTIFY API CREDENTIALS HAVE BEEN REMOVED FOR SECURITY PURPOSES - If you run the program it will give you an error, because it will need these numbers.
CLIENT_ID= '***'
CLIENT_SECRET= '***'

In [4]:
token = oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

<ipython-input-4-5c2b7eb2a594>:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  cache_token = token.get_access_token()


USING A FUNCTION TO EXTRACT THE DATA FROM SPOTIFY - FOLLOWING THIS TUTORIAL: https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6

In [5]:
def analyze_playlist(creator, playlist_id):
    
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df


In [6]:
playlist_df = analyze_playlist('Spotify', "37i9dQZF1DX7Jl5KP2eZaS")

In [74]:
playlist_df 

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.514,0.730,1,-5.934,1,0.0598,0.000095,0.0897,0.3340,171.005,200040,4
1,Tones And I,Dance Monkey,Dance Monkey,1rgnBhdG2JDFTbYkYRZAku,0.825,0.593,6,-6.401,0,0.0988,0.000161,0.1700,0.5400,98.078,209755,4
2,Roddy Ricch,Please Excuse Me For Being Antisocial,The Box,0nbXyq5TXYPCO7pr3N8S4I,0.896,0.586,10,-6.687,0,0.0559,0.000000,0.7900,0.6420,116.971,196653,4
3,SAINt JHN,Roses (Imanbek Remix),Roses - Imanbek Remix,2Wo6QQD1KMDWeFkkjLqwx5,0.785,0.721,8,-5.457,1,0.0506,0.004320,0.2850,0.8940,121.962,176219,4
4,Dua Lipa,Future Nostalgia,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,0.793,0.793,11,-4.521,0,0.0830,0.000000,0.0951,0.6790,123.950,183290,4
5,DaBaby,BLAME IT ON BABY,ROCKSTAR (feat. Roddy Ricch),7ytR5pFWmSjzHJIeQkgog4,0.746,0.690,11,-7.956,1,0.1640,0.000000,0.1010,0.4970,89.977,181733,4
6,Harry Styles,Fine Line,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,0.548,0.816,0,-4.209,1,0.0465,0.000000,0.3350,0.5570,95.390,174000,4
7,Powfu,death bed (coffee for your head),death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.726,0.431,8,-8.765,0,0.1350,0.000000,0.6960,0.3480,144.026,173333,4
8,Trevor Daniel,Nicotine,Falling,2rRJrJEo19S2J82BDsQ3F7,0.784,0.430,10,-8.756,0,0.0364,0.000000,0.0887,0.2360,127.087,159382,4
9,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,Someone You Loved,7qEHsqek33rTcFNT9PFqLf,0.501,0.405,1,-5.679,1,0.0319,0.000000,0.1050,0.4460,109.891,182161,4


 CLEANING UP THE PLAYLIST SO THAT WE JUST END UP WITH TRACK_NAME, ARTIST AND ALBUM

In [75]:
df = playlist_df[['track_name','artist','album']]

In [76]:
#CLEANING UP SONG AND ALBUM NAME SO THAT IT'S EASIER FOR THE QUIZZ (Erasing '-' and '()')
import re
tracks = []

for t in df['track_name']:
    if re.search(r'\(',t):
        t2 = t.split('(')
        tracks.append(t2[0].strip()) #Using strip for erasing empty spaces at the end of string
    elif re.search('-',t):
        t2 = t.split('-')
        tracks.append(t2[0].strip())
    else:
        t = t
        tracks.append(t)
df['track_name'] = tracks

albums = []
for t in df['album']:
    if re.search(r'\(',t):
        t = t.split('(')
        albums.append(t[0].strip())
    elif re.search('-',t):
        t= t.split('-')
        albums.append(t[0].strip())
    else:
        t = t
        albums.append(t)
df['album'] = albums

<ipython-input-76-0706be0e6d30>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['track_name'] = tracks
<ipython-input-76-0706be0e6d30>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['album'] = albums


SAVING THE FINAL DATA SET INTO A CSV

In [77]:
df.to_csv('2020_MUSIC.csv', index = False)

In [78]:
df = pd.read_csv('2020_MUSIC.csv')

In [79]:
df

,track_name,artist,album
0,Blinding Lights,The Weeknd,After Hours
1,Dance Monkey,Tones And I,Dance Monkey
2,The Box,Roddy Ricch,Please Excuse Me For Being Antisocial
3,Roses,SAINt JHN,Roses
4,Don't Start Now,Dua Lipa,Future Nostalgia
5,ROCKSTAR,DaBaby,BLAME IT ON BABY
6,Watermelon Sugar,Harry Styles,Fine Line
7,death bed,Powfu,death bed
8,Falling,Trevor Daniel,Nicotine
9,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent
